**Домашнее задание необходимо предоставить в формате ссылки на Google Collab/Jupiter Notebook с вашими действиями и ключевыми выводами.**

- 21 августа 23:59 — мягкий дедлайн, 28 августа 23:59 — жесткий дедлайн  
- До мягкого дедлайна за работу можно получить 10 баллов, после — 5  
- Работы, отправленные после 28 августа, могут быть проверены преподавателями до конца курса в формате зачет/не зачет  

### Домашнее задание

**Задание 1. Подготовка данных и модели — 2 балла**

1. **_Выбор датасета — 0.5 балла_**
   - Выберите и загрузите датасет для задачи генерации текста  
   - Подходящие варианты:
     - Текстовые корпусы на платформе Hugging Face  
     - Ваши собственные данные (при условии, что они доступны и соответствуют задаче)  

2. **_Выбор предобученной модели — 0.5 балла_**
   - Выберите подходящую (пока что до 350 млн параметров) предобученную языковую модель на Hugging Face Model Hub  
   - Для русскоязычных задач можно рассмотреть:  
     `ai-forever/rugpt3small_based_on_gpt2`,  
     `ai-forever/rugpt3medium_based_on_gpt2`,  
     `cointegrated/rut5-base и другие`  
   - Для англоязычных задач:  
     `facebook/opt-350m`,  
     `EleutherAI/gpt-neo-125M и другие`  

3. **_Предварительная обработка данных — 1 балл_**
   - Реализуйте токенизацию данных с использованием выбранного токенизатора  
   - Выберите оптимальную длину контекста (context_length) для вашей задачи  
   - Подготовьте данные для обучения (разделение на обучающую и валидационную выборки — если есть в датасете, либо можно сделать через `train_test_split`, батчинг)  

**Задание 2. Дообучение модели — 3 балла**

1. **_Предварительная оценка качества — 0.5 балла_**
   - Проверьте качество генерации перед дообучением на нескольких примерах  

2. **_Настройка процесса обучения — 2 балла_**
   - Настройте параметры обучения (learning rate, batch size, количество эпох)  
   - Обучите модель на вашем датасете в режиме полного дообучения  

3. **_Оценка качества обучения — 0.5 балла_**
   - Проверьте качество генерации после дообучения на нескольких примерах  

**Задание 3. Сэмплирование — 5 баллов**

1. **_Эксперименты с сэмплированием — 3 балла_**
   - Выберите и протестируйте как минимум 4 различных комбинации параметров сэмплирования:
     - Greedy decoding (`do_sample=False`)  
     - Temperature sampling (разные значения `temperature` — меньше 1, 1, больше 1...)  
     - Top-k sampling (разные значения `top_k`)  
     - Top-p (nucleus) sampling (разные значения `top_p`)  
     - Min-p sampling (`min_p`)  
     - Beam search (`num_beams > 1`)  
   - NB: Обратите внимание на взаимодействие параметров (например, как `top_p` работает с `top_k`)  

2. **_Анализ результатов — 2 балла_**
   - Проведите анализ влияния получившихся комбинаций на:
     - Грамматичность и связность генерируемого текста  
     - Креативность и разнообразие генерации  
     - Соответствие стилю исходного датасета  
     - Наличие повторений и прочих артефактов генерации  
   - Определите оптимальные параметры для вашей задачи генерации и обоснуйте ваш выбор  